In [1]:
# importiere Library und build Instanz

from camera_roboter import CameraRoboter
from math import pi
import rospy

CameraRoboterI = CameraRoboter(load_camera=True)

[ INFO] [1685988928.438456424]: Loading robot model 'panda'...
[ INFO] [1685988929.725086585]: Ready to take commands for planning group panda_arm.
Initialize workspace...
============ Planning frame: panda_link0
============ End effector link: panda_link8
============ Available Planning Groups: ['panda_arm']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  name: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  position: [-0.00030285222559611784, -0.7859434514154087, -0.00027212492192736377, -2.3557003563314836, -0.00010875958655430233, 1.5718685730524202, -0.7848677676288688]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision

In [2]:
# prüfe scharfes bild und Position

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.move_ready_pose()
    CameraRoboterI.get_video_stream()
finally:
    CameraRoboterI.Camera.close()

# labeln jeder Stelle der Schweißnah

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.calibration(0.016, 5)
finally:
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()

initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0
initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0


In [2]:
# eingebe Info der Probe

dict_info = dict()

dict_info['id'] = "5_004"
dict_info['speed'] = "500"
dict_info['rotate_speed'] = "2100"
dict_info['date'] = "06.11.19"

training_label = list(range(6))
training_label[0] = 1
training_label[1] = 1
training_label[2] = 1
training_label[3] = 1
training_label[4] = 1
training_label[5] = 1

dict_info['training_label'] = training_label
CameraRoboterI.DataManager.set_basic_info(dict_info)
dict_info

{'id': '5_004',
 'speed': '500',
 'rotate_speed': '2100',
 'date': '06.11.19',
 'training_label': [1, 1, 1, 1, 1, 1]}

In [3]:
# Hauptprogramm

try:
    # initialize Kamera
    CameraRoboterI.Camera.init_camera()

    # setzt Heilligkeitstufe
    brightness_class = [30, 50, 70, 100]

    # jeder Heilligkeitstufe kommplette Prozess laufen
    for i in range(len(brightness_class)):

        # ---manuell Heilligkeit einstellen---
        _ = input("Bitte stellt Heilligkeit manuell auf {} ein!".format(brightness_class[i]))

        # ---linear Bewegung---
        CameraRoboterI.DataManager.set_brightness(brightness_class[i])
        CameraRoboterI.move_ready_pose(save_image=True)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

        # ---drehe um z-Achse---
        ## kleine Winkel von pi/100
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(pi/100)
        [j0, j1, j2, j3, j4, j5, j6] = [-0.00044218823784680534,
                                        -0.2402933855098529,
                                        -0.0014397267436032388,
                                        -2.5626248980405038,
                                        -0.0019084416925907136,
                                        2.3282285336388484,
                                        -0.754219668120121]
        CameraRoboterI.move_ready_pose()
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True, image_index=0)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

        ## etwa größe Winkel von pi/50
        [j0, j1, j2, j3, j4, j5, j6] = [-0.0007674324706054823,
                                        -0.2402885262255083,
                                        -0.0033156575211895664,
                                        -2.5626367920848874,
                                        -0.0025284310955359162,
                                        2.328318544066852,
                                        -0.724393343723154]
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(pi/50)
        CameraRoboterI.move_ready_pose()
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True, image_index=0)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

        # setzt Winkel wieder zurück auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(0)

        # ---drehe um x-Achse--- um pi/25
        [j0, j1, j2, j3, j4, j5, j6] = [-0.08354183469843446,
                                        -0.229609848980318,
                                        0.20267543657217285,
                                        -2.546915491672936,
                                        -0.11250449655674946,
                                        2.307769107599535,
                                        -0.594767860662422]
        CameraRoboterI.move_ready_pose()
        angle = CameraRoboterI.DataManager.set_rotate_angle_x(pi/25)
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True, image_index=0)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

        # setzt Winkel wieder zuruck auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_x(0)

        # ---unscharfe manuell einstellen---
        ## manuell einstellen
        if brightness_class[i] == 50:
            CameraRoboterI.move_ready_pose()
            CameraRoboterI.get_video_stream()

            ## linear Bewegung
            CameraRoboterI.DataManager.set_zoom_blur(1)
            CameraRoboterI.move_ready_pose(save_image=True)
            CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

            # setzt blur wieder zurück
            CameraRoboterI.move_ready_pose()
            CameraRoboterI.get_video_stream()
            CameraRoboterI.DataManager.set_zoom_blur(0)

finally:
    # schließe Kamera und zurück zur Initialposition
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()




initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
image is saved als name: images/5_004_500_2100/05.06.2023_20:15:54_0.43_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:15:55_0.45_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:15:56_0.46_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:15:57_0.48_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:15:59_0.49_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:16:00_0.51_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:16:02_0.43_30_0_0.03_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:16:03_0.45_30_0_0.03_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_20:16:04_0.46_30_0_0.03_0_1.jpg
image is saved als n